In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

from noteify.core.datasets import (MusicNetDataset, MusicNetDatasetProcessed, MusicAugmentor,
                                   MusicNetSampler, get_musicnet_dataloader)
from noteify.core.training import make_optimizer, train_model
from noteify.core.models import TranscriptionNN
from noteify.core.utils import plot_audio, plot_roll_info

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("Warning: Could not find GPU! Using CPU only")

C:\Users\Shahir\anaconda3\envs\py3-music\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
data_dir = r"D:\Users\Shahir\Documents\Code Repositories\music-transcription\musicnet"
save_dir = r"D:\Users\Shahir\Documents\Code Repositories\music-transcription\weights"

In [4]:
raw_dataset_train = MusicNetDataset(data_dir, download=True, train=True)
raw_dataset_test = MusicNetDataset(data_dir, download=True, train=False)

In [6]:
batch_size = 256

In [7]:
dataset_train = MusicNetDatasetProcessed(raw_dataset_train, augmentor=MusicAugmentor())
sampler_train = MusicNetSampler(dataset_train, batch_size)
dataloader_train = get_musicnet_dataloader(dataset_train, sampler_train)

dataset_test = MusicNetDatasetProcessed(raw_dataset_test)
sampler_test = MusicNetSampler(dataset_test, batch_size)
dataloader_test = get_musicnet_dataloader(dataset_test, sampler_test)

dataloaders = {'train': dataloader_train, 'test': dataloader_test}

In [ ]:
model = TranscriptionNN()
model = model.to(device)
# past_train_weights = os.path.join(save_dir, "Weights.pckl")
# model.load_state_dict(torch.load(past_train_weights))

In [ ]:
optimizer = make_optimizer(model, lr=0.001)
scheduler = make_scheduler(optimizer, [30, 40], gamma=0.2)

In [ ]:
tracker = train_model(device=device,
                      model=model,
                      dataloaders=dataloaders,
                      optimizer=optimizer,
                      lr_scheduler=scheduler,
                      save_dir=save_dir,
                      save_best=True,
                      save_all=False,
                      num_epochs=num_epochs)